In [1]:
import pandas as pd
import requests as httpr
from pprint import pprint

from splinter import Browser
from bs4 import BeautifulSoup

# @NOTE: Replace the path with your actual path to the chromedriver
#executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
executable_path = {'executable_path': 'chromedriver.exe'}



In [2]:
#for the sake of performance and my own learning I will be using the Request library for retrieving the HTML since it will not need to laod images.

#retrieve nasa news items
browser = Browser('chrome', **executable_path, headless=False)
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

article_title = soup.find("div", class_="content_title").get_text()
article_body = soup.find("div", class_="article_teaser_body").get_text()
print(article_title)
print(article_body)

browser.quit()


NASA's Mars InSight Flexes Its Arm
Now unstowed, the spacecraft's robotic arm will point a camera located on its elbow and take images of the surroundings.


In [3]:
#retrieve latest nasa mars image url
base_url = "https://www.jpl.nasa.gov"
url = base_url + "/spaceimages/?search=&category=Mars"
http_result = httpr.get(url)

soup = BeautifulSoup(http_result.content, "html.parser")

image_page_url = soup.find("a", class_="button fancybox")['data-link']
new_url = base_url + image_page_url

http_result = httpr.get(new_url)
soup = BeautifulSoup(http_result.content, "html.parser")

full_img_page_url = base_url + soup.find("img", class_="main_image")['src']
print(full_img_page_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14934_hires.jpg


In [4]:
#get the latest tweet from the Mars Weather account
url = "https://twitter.com/marswxreport?lang=en"
http_result = httpr.get(url)
quoted_tweet_optional = ""

soup = BeautifulSoup(http_result.content, "html.parser")

tweet_text = soup.find("p", class_="tweet-text").get_text()
if soup.find("p", class_="QuoteTweet-text") != None:
    quoted_tweet_optional = soup.find("p", class_="QuoteTweet-text").get_text()
print(tweet_text)
print(quoted_tweet_optional)


Well done! That 30 minutes of EDL dust settling was very effective. Shame #InSight can’t act as a supercharger for @marsrovers Oppy, she sure could use a boost right now.https://twitter.com/NASAInSight/status/1068661716756516864 …



In [10]:
#retrieve the facts table and load to a dataframe
url = "http://space-facts.com/mars/"

mars_facts_df = pd.DataFrame(pd.read_html(url)[0])
mars_facts_df.columns = ["Feature", "Value"]
html_table = mars_facts_df.to_html(classes='table-striped')
pprint(html_table)

('<table border="1" class="dataframe table-striped">\n'
 '  <thead>\n'
 '    <tr style="text-align: right;">\n'
 '      <th></th>\n'
 '      <th>Feature</th>\n'
 '      <th>Value</th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <th>0</th>\n'
 '      <td>Equatorial Diameter:</td>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>1</th>\n'
 '      <td>Polar Diameter:</td>\n'
 '      <td>6,752 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>2</th>\n'
 '      <td>Mass:</td>\n'
 '      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>3</th>\n'
 '      <td>Moons:</td>\n'
 '      <td>2 (Phobos &amp; Deimos)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>4</th>\n'
 '      <td>Orbit Distance:</td>\n'
 '      <td>227,943,824 km (1.52 AU)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>5</th>\n'
 '      <td>Orbit Period:</td>\n'
 '      <td>687 days (1.9 years)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <

In [6]:
base_url = "https://astrogeology.usgs.gov"
url = base_url + "/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
http_result = httpr.get(url)

soup = BeautifulSoup(http_result.content, "html.parser")
hemisphere_img_info = []
original_img_url = ''

for link in soup.find_all("a", class_="product-item"):
    image_text = link.get_text()
    
    url = base_url + link["href"]
    http_result = httpr.get(url)
    image_soup = BeautifulSoup(http_result.content, "html.parser")
    image_url = base_url + image_soup.find("img", class_="wide-image")["src"]
    
    for img_link in image_soup.find_all("a"):
        if (img_link.get_text() == 'Original'):
            original_img_url = base_url + img_link['href']
            
    img_info = {"title": image_text,
               "url_jpg": image_url,
               "original_image_url": original_img_url}
    
    hemisphere_img_info.append(img_info)
    
pprint(hemisphere_img_info)




[{'original_image_url': 'https://astrogeology.usgs.govhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
  'title': 'Cerberus Hemisphere Enhanced',
  'url_jpg': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'original_image_url': 'https://astrogeology.usgs.govhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
  'title': 'Schiaparelli Hemisphere Enhanced',
  'url_jpg': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'original_image_url': 'https://astrogeology.usgs.govhttp://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
  'title': 'Syrtis Major Hemisphere Enhanced',
  'url_jpg': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'original_image_url': 'https://astrogeology.usgs.govhttp

In [18]:
    import pymongo
    import scrape_mars as sm
    from flask import jsonify
    
    # Query all passengers
    latest_mars_info = sm.scrape()

    # Create connection variable
    conn = 'mongodb://localhost:27017'

    # Pass connection to the pymongo instance.
    client = pymongo.MongoClient(conn)

    # Connect to a database. Will create one if not already available.
    db = client.mars_db

    # Drops collection if available to remove duplicates
    db.mars.drop()

    # Creates a collection in the database and inserts two documents
    db.mars.insert_many(latest_mars_info)

    list(db.mars.find())

NASA InSight Lander 'Hears' Martian Winds 
Vibrations picked up by two spacecraft instruments have provided the first sounds of Martian wind.


[{'_id': ObjectId('5c0ac538b0d7b845807eab04'),
  'article_info': {'article_title': "NASA InSight Lander 'Hears' Martian Winds ",
   'article_body': 'Vibrations picked up by two spacecraft instruments have provided the first sounds of Martian wind.'}},
 {'_id': ObjectId('5c0ac538b0d7b845807eab05'),
  'feature_image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18297_hires.jpg'},
 {'_id': ObjectId('5c0ac538b0d7b845807eab06'),
  'weather_tweet': 'Listen to northwesterly winds blowing at 10-15 mph across the deck of the @NASAInSight landerhttps://www.youtube.com/watch?v=o3cxuIsEFRM\xa0…'},
 {'_id': ObjectId('5c0ac538b0d7b845807eab07'),
  'facts_table': '<table border="1" class="dataframe table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Feature</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n  